In [ ]:
import requests  # requests 라이브러리를 사용하여 HTTP 요청을 보냄  
import re
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_ENDPOINT = os.getenv("OPENAI_ENDPOINT")
OPENAI_APIKEY = os.getenv("OPENAI_APIKEY")
AISEARCH_ENPOINT = os.getenv("AISEARCH_ENPOINT")
AISEARCH_APIKEY = os.getenv("AISEARCH_APIKEY")

# Azure OpenAI 서비스의 엔드포인트 URL  
endpoint = OPENAI_ENDPOINT 
# API 키를 설정. 반드시 보안을 유지해야 하며, 실제 개발 환경에서는 환경 변수에 저장하는 것이 좋음  
api_key = OPENAI_APIKEY

# Azure AI Search 서비스의 엔드포인트 URL  
ai_search_endpoint =   AISEARCH_ENPOINT
# Azure AI Search 서비스의 API 키  
ai_search_api_key = AISEARCH_APIKEY
# Azure AI Search에서 사용할 인덱스 이름  
ai_search_index = "travel-index-microsoft2"
ai_search_semantic = "travel-semantic"

def request_gpt(prompt):
    
    # HTTP 요청에 필요한 헤더 설정  
    # Content-Type은 요청 본문이 JSON 형식임을 나타냄  
    # api-key는 Azure OpenAI 서비스의 인증에 사용됨  
    headers = {  
        "Content-Type": "application/json",  
        "api-key": api_key
    }
    
    # HTTP 요청의 본문 데이터  
    # messages 리스트에는 대화의 역할(role)과 내용(content)이 포함됨  
    # "system" 역할은 모델의 행동을 정의하며, 여기서는 "남해 여행 전문가"로 설정  
    # "user" 역할은 사용자가 입력한 메시지를 나타냄  
    body = {  
        "messages": [  
            {  
                "role": "system",  # 시스템 역할: 모델의 컨텍스트 설정  
                "content": "너는 남해 여행 전문가야."  # 모델에게 특정 역할을 부여  
            },  
            {  
                "role": "user",  # 사용자 역할: 사용자의 요청  
                "content": prompt  # 사용자가 알고 싶은 내용 (남해 관광지 3곳)  
            }  
        ],  
        "temperature": 0.7,  # 응답의 창의성 정도를 조절 (0.0은 보수적, 1.0은 매우 창의적)  
        "top_p": 0.95,  # 확률 분포에서 상위 p%를 선택하여 응답 생성  
        "max_tokens": 800,  # 응답에 사용할 최대 토큰 수 (토큰은 단어 및 기호 단위)  
        "data_sources": [
            {
            "type": "azure_search",
            "parameters": {
                "endpoint": ai_search_endpoint,
                "index_name": ai_search_index,
                "semantic_configuration": ai_search_semantic,
                "query_type": "semantic",
                "fields_mapping": {},
                "in_scope": True,
                "filter": None,
                "strictness": 3,
                "top_n_documents": 5,
                "authentication": {
                    "type": "api_key",
                    "key": ai_search_api_key
                },
                "key": ai_search_api_key,
                
            }
            }
        ],
    }  
    
    # POST 요청을 보내고 응답 받기  
    response = requests.post(endpoint, headers=headers, json=body)  

    if response.status_code == 200:

        # 응답을 JSON 형식으로 파싱  
        response_json = response.json()  
        
        # 모델이 생성한 메시지 추출  
        message = response_json['choices'][0]['message']  
        citaiton_list = message['context']['citations']
        # 역할(role)과 내용(content) 분리  
        role = message['role']  # 메시지의 역할 (예: assistant)  
        content = message['content']  # 메시지의 내용 (예: 남해 관광지 정보)          
        content = re.sub(r'\[doc(\d+)\]', r'[참조 \1]', content)
        return content, citaiton_list
    else:
        return "", list()
  
content, citation_list = request_gpt("남해 관광지 3곳만 알려줘")
  
# Markdown 형식으로 응답 내용을 출력 (IPython 환경에서 사용 가능)  
from IPython.display import Markdown  
display(Markdown(content))  

for index in range(len(citation_list)):
    c = citation_list[index]
    print("[참조 {}]".format(index + 1))
    print(c['content'])

남해의 유명한 관광지 세 곳을 추천드립니다:

1. **천여 그루의 방풍림**:
   - 위치: 경상남도 남해군 삼동면 동부대로1030번길 59
   - 특징: 천여 그루의 방풍림으로 조성된 울창한 숲이며, 남해 바다의 정취를 느낄 수 있는 곳입니다 [참조 1].

2. **남해 상상양떼목장**:
   - 위치: 경상남도 남해군 설천면 설천로775번길 364
   - 특징: 푸른 초원의 양떼들과 함께하는 목장 체험을 즐길 수 있는 곳입니다 [참조 3].

3. **망운산**:
   - 위치: 경상남도 남해군 서면 남서대로2240번길 412-18
   - 특징: 남해의 최고 높이를 자랑하는 산으로, 아름다운 풍경을 감상할 수 있습니다 [참조 4].

[참조 1]
남해군 삼동면 동부대로1030번길 59
2천여 그루의 방풍림으로 조성된 울창한 수림이 초승달 모양을 그리며 장관을 이루고 있는 곳으로 숲과 어우러진 몽돌해안은 남해 바다의 정취가 물씬 풍기는 독특함을 자랑한다.
Y
34.7958875
128.0504234
남해 물건리 방조어부림과 물미해안
055-860-8631
봄, 여름, 가을, 겨울, 자가용/렌트, 버스, 친구, 키즈, 연인, 우정女, 우정男, 효도, 나홀로, 단체
관광지
천연기념물 제150호
[참조 2]
경상남도 남해군 남면로 674
몇 번을 세어 보았지만 찾을 수 없어 포기하고 집으로 돌아기가 위해 벗어둔 삿갓을 들었더니 삿갓 아래 한 배미가 있었다는 이야기는 작은 땅도 논으로 만들어 경작했다는 남해인들의 억척 같은 삶을 우화적으로 표현하고 있다.
남해가천마을 다랑이논
[참조 3]
경상남도 남해군 설천면 설천로775번길 364
굽이굽이 구불구불 비탈진 인생도 생애 어느 날만큼은 탁 트인 초원에서 모든 상념을 내려놓고 쉬어 가야 하지 않을까. 탁 트인 드넓은 들판. 푸른 초록의 숲과 들, 눈 앞에 펼쳐진 시원한 바다. 곳곳에 우리를 둘러싼 귀여운 양들의 산책까지. 이 모든 걸 만날 수 있는 남해상상양떼목장이 설천면 구두산 일대에 펼쳐져 있다.
Y
http://yangttefarm.com/
34.9361826
127.8906311
남해 상상양떼목장 편백숲
055-862-5300
봄, 여름, 가을, 겨울, 자가용/렌트, 친구, 키즈, 연인, 우정女, 우정男, 효도
관광지
[참조 4]
경상남도 남해군 서면 남서대로2240번길 412-18
786m, 남해 최고 높이를 자랑하는 망운산. 그러나 남해안 제1의 명산인 금산에 가려져 진가가 꼭꼭 숨겨진 곳이다. 망운산을 오르는 사람은 이곳이 알려지길 두려워한다. 깨끗한 풍모, 드넓은 기상, 아는 자만 오르리라. 금산이 남해를 찾는 손님들의 산이라면, 망운산은 남해인들이 가장 아끼는 늠름한 기상이다.
Y
34.8458603
127.8461366
망운산
055